Проективное преобразование


In [24]:
from sys import float_info
from core import CustomImage, ObjModel, face_angle_cos, barisentrik_coordinates
from names import FOX, DEER


def draw_visible_faces(obj: ObjModel, _img: CustomImage, scale: tuple[float, float], shift: tuple[float, float, float]):
    z_buffer = [[float_info.max for _ in range(_img.width())] for _ in range(_img.height())]
    img_x, img_y = _img.width() / 2, _img.height() / 2
    ax, ay = scale
    tx, ty, tz = shift
    for f in obj.faces():
        v0, v1, v2 = f

        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2

        v0 = ax * (x0 + tx) + img_x * (z0 + tz), ay * (y0 + ty) + img_y * (z0 + tz), z0 + tz
        v1 = ax * (x1 + tx) + img_x * (z1 + tz), ay * (y1 + ty) + img_y * (z1 + tz), z0 + tz
        v2 = ax * (x2 + tx) + img_x * (z2 + tz), ay * (y2 + ty) + img_y * (z2 + tz), z0 + tz

        f = v0, v1, v2

        cos = face_angle_cos(f)
        if cos >= 0:
            continue

        v0 = x0, y0
        v1 = x1, y1
        v2 = x2, y2

        val = round(255 * -cos)
        color = (val, val, val)

        x_min = round(max(min(x0, x1, x2), 0))
        y_min = round(max(min(y0, y1, y2), 0))
        x_max = round(min(max(x0, x1, x2), _img.width() - 1))
        y_max = round(min(max(y0, y1, y2), _img.height() - 1))

        for i in range(x_min, x_max + 1):
            for j in range(y_min, y_max + 1):
                v = (i, j)
                l0, l1, l2 = barisentrik_coordinates(v, v0, v1, v2)
                if l0 > 0 and l1 > 0 and l2 > 0:
                    z = l0 * z0 + l1 * z1 + l2 * z2
                    if z < z_buffer[i][j]:
                        _img.set(v, color)
                        z_buffer[i][j] = z


t = 1, -2, 15.0

fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

img = CustomImage(1000, 1000)
draw_visible_faces(fox_obj, img, (1000, 1000), t)
img.save('pictures/faces/fox_projection.png')

img = CustomImage(1500, 1500)
draw_visible_faces(deer_obj, img, (10, 10), t)
img.save('pictures/faces/deer_projection.png')